In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [94]:
events = pd.read_json('events\events_World_Cup.json')
matches = pd.read_json('matches\matches_World_Cup.json')
teams = pd.read_json('teams.json')

In [96]:
events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,ev_success
0,8,Simple pass,[{'id': 1801}],122671,"[{'y': 50, 'x': 50}, {'y': 53, 'x': 35}]",2057954,Pass,16521,1H,1.656214,85,258612104,1.0
1,8,High pass,[{'id': 1801}],139393,"[{'y': 53, 'x': 35}, {'y': 19, 'x': 75}]",2057954,Pass,16521,1H,4.487814,83,258612106,1.0
2,1,Air duel,"[{'id': 703}, {'id': 1801}]",103668,"[{'y': 81, 'x': 25}, {'y': 83, 'x': 37}]",2057954,Duel,14358,1H,5.937411,10,258612077,1.0
3,1,Air duel,"[{'id': 701}, {'id': 1802}]",122940,"[{'y': 19, 'x': 75}, {'y': 17, 'x': 63}]",2057954,Duel,16521,1H,6.406961,10,258612112,0.0
4,8,Simple pass,[{'id': 1801}],122847,"[{'y': 17, 'x': 63}, {'y': 15, 'x': 71}]",2057954,Pass,16521,1H,8.562167,85,258612110,1.0


In [95]:
# adaptando coluna 'tags' para conseguir interpretar se evento foi bem sucedido ou não. 'id' = 1801 ou 1802
def converting_tags(aux):
    new = pd.Series()
    try:
        new = list(aux[-1].values())[0]
    except IndexError:
        return None

    return new

events['ev_success'] = events['tags'].map(converting_tags)
events['ev_success'] = events['ev_success'].replace([1801, 1802], [1, 0])

C:\Users\eduar\AppData\Local\Temp/ipykernel_26556/300082212.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new = pd.Series()


In [100]:
#contar quantos subEvents = 'Save attempt' com tag = 1801(ev_success = 1) para todos os (match, team)

a = events[(events['subEventName'] == 'Save attempt') & events['ev_success'] == 1].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
a.columns = a.columns.droplevel(0)
a = a.rename(columns={'count':'gk_save'})
a.head()

#trocar subrows por novas colunas


#fazer o mesmo do df das redes para esse aqui (criar df com features)

gk_save
matchId teamId         
2057954 16521         1
2057955 15670         3
        16129         1
2057956 16129         1
2057957 15670         3

In [101]:
matches.head()

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,groupName,referees,duration,competitionId
0,Played,4165368,0,"{'9598': {'scoreET': 0, 'coachId': 122788, 'si...",10078,2018-07-15 15:00:00,4418,Olimpiyskiy stadion Luzhniki,2058017,"France - Croatia, 4 - 2",2018-07-15 17:00:00-02:00,,"[{'refereeId': 378051, 'role': 'referee'}, {'r...",Regular,28
1,Played,4165367,0,"{'2413': {'scoreET': 0, 'coachId': 276858, 'si...",10078,2018-07-14 14:00:00,5629,Stadion Krestovskyi,2058016,"Belgium - England, 2 - 0",2018-07-14 16:00:00-02:00,,"[{'refereeId': 380599, 'role': 'referee'}, {'r...",Regular,28
2,Played,4165366,0,"{'2413': {'scoreET': 1, 'coachId': 276858, 'si...",10078,2018-07-11 18:00:00,9598,Olimpiyskiy stadion Luzhniki,2058015,"Croatia - England, 2 - 1 (E)",2018-07-11 20:00:00-02:00,,"[{'refereeId': 384995, 'role': 'referee'}, {'r...",ExtraTime,28
3,Played,4165366,0,"{'5629': {'scoreET': 0, 'coachId': 8605, 'side...",10078,2018-07-10 18:00:00,4418,Stadion Krestovskyi,2058014,"France - Belgium, 1 - 0",2018-07-10 20:00:00-02:00,,"[{'refereeId': 378232, 'role': 'referee'}, {'r...",Regular,28
4,Played,4165365,0,"{'14358': {'scoreET': 2, 'coachId': 264893, 's...",10078,2018-07-07 18:00:00,9598,Olimpiyskiy Stadion Fisht,2058012,"Russia - Croatia, 2 - 2 (P)",2018-07-07 20:00:00-02:00,,"[{'refereeId': 378204, 'role': 'referee'}, {'r...",Penalties,28
